In [1]:
!nvidia-smi
from google.colab import drive
drive.mount('/content/drive')
!ls drive/'My Drive'/'Colab Notebooks'//bad_chatbot
!pip install -r drive/'My Drive'/'Colab Notebooks'/bad_chatbot/Bert_Gpt_Chatbot/requirements.txt
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
import sys
sys.path.append('drive/My Drive')
sys.path.append('drive/My Drive/Colab Notebooks/')
sys.path.append('drive/My Drive/Colab Notebooks/bad_chatbot')
!pip install sentencepiece
!pip install transformers==4.16.2

Mon Feb 28 15:15:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm

from Bert_Gpt_Chatbot.dataloader.dataloader import GPT2Dataset
from Bert_Gpt_Chatbot.model.kogpt2_model import GPT2Chat
torch.cuda.is_available()
ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)

In [5]:
root_path='drive/My Drive/Colab Notebooks/bad_chatbot/Bert_Gpt_Chatbot'

chatbot_file = pd.read_csv(f'{root_path}/input/datasets/KoGPT_chatbot_30000.csv')
chatbot_file.dropna(inplace=True)
chatbot_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30260 entries, 0 to 30263
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       30260 non-null  object
 1   A       30260 non-null  object
dtypes: object(2)
memory usage: 709.2+ KB


In [6]:

epochs = 1
batch_size = 8
Sneg = -1e18
learning_rate = 3e-5

model = GPT2Chat()
model.to(device)

train_dataset = GPT2Dataset(chatbot_file)


def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=0, shuffle=True, collate_fn=collate_batch,)


no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss(reduction="none")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [7]:
checkpoint_path =f"{root_path}/output"
save_ckpt_path = f'{checkpoint_path}/model/cp_chatbot.pt'

pre_epoch, pre_loss, train_step = 0, 0, 0
if os.path.isfile(save_ckpt_path):
    checkpoint = torch.load(save_ckpt_path, map_location=device)
    
    pre_epoch = checkpoint['Epoch']
    model.load_state_dict(checkpoint['State_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

    # pre_loss = checkpoint['Loss']
    # train_step =  checkpoint['Train_no']
    # total_train_step =  checkpoint['Total_train_step']

    print(f"load pretrain from: {save_ckpt_path}, epoch={pre_epoch}")  #, loss={pre_loss}\n")
    # best_epoch += 1

In [8]:
minimum_loss = 100

model.train()
for epoch in range(epochs):
    for batch_idx, (token_ids, mask, label) in enumerate(tqdm(train_dataloader)):
        token_ids = token_ids.to(device)
        mask = mask.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        out = model(token_ids).logits
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss_2 = avg_loss
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
    
    
    state = {'Epoch': epoch,
             'State_dict': model.state_dict(),
             'optimizer': optimizer.state_dict()}
    torch.save(state, f'{checkpoint_path}/model/cp_chatbot.pt')
    print('epoch:', epoch, 'loss:', avg_loss)

    if avg_loss_2 < minimum_loss :
        model.eval()
        torch.save(model, f'{checkpoint_path}/model/KoGPT_chatbot_{avg_loss_2}.pt')
        minimum_loss = avg_loss_2

100%|██████████| 3783/3783 [12:20<00:00,  5.11it/s]


epoch: 0 loss: tensor(20.1814, device='cuda:0', grad_fn=<DivBackward0>)


In [ ]:
### 참고 코드

# checkpoint_path =f"{root_path}/checkpoint"
# save_ckpt_path = f"{checkpoint_path}/test_GPT.pth"


# n_epoch = 5         # Num of Epoch
# batch_size = 1      # 배치 사이즈
# ctx = "cuda" if torch.cuda.is_available() else "cpu"
# device = torch.device(ctx)
# save_step = 100 # 학습 저장 주기

# learning_rate = 5e-5  # Learning Rate

# dataset= WellnessAutoRegressiveDataset(data_path)
# train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# model = DialogKoGPT2()
# model.to(device)


# loss_fct = torch.nn.CrossEntropyLoss(ignore_index=3)
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# ###################
# pre_epoch, pre_loss, train_step = 0, 0, 0
# if os.path.isfile(save_ckpt_path):
#     checkpoint = torch.load(save_ckpt_path, map_location=device)
#     pre_epoch = checkpoint['epoch']
#     pre_loss = checkpoint['loss']
#     train_step =  checkpoint['train_no']
#     total_train_step =  checkpoint['total_train_step']
#     model.load_state_dict(checkpoint['model_state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

#     print(f"load pretrain from: {save_ckpt_path}, epoch={pre_epoch}")  #, loss={pre_loss}\n")
#     # best_epoch += 1


# ###################

# losses =[]
# for epoch in range(n_epoch):
#     count = 0
#     with tqdm(total=len(train_loader), desc=f"Train({epoch})") as pbar:
#         for i, data in enumerate(train_loader):
#             optimizer.zero_grad()
#             data = torch.stack(data)  # list of Tensor로 구성되어 있기 때문에 list를 stack을 통해 변환해준다.
#             data = data.transpose(1, 0)
#             data= data.to(ctx)

#             outputs = model(data, labels=data)
#             _, logits = outputs[:2]

#             shift_logits = logits[..., :-1, :].contiguous()
#             shift_labels = data[..., 1:].contiguous()

#             loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
#             loss.backward()
#             optimizer.step()

#             losses.append(loss.item())


#             # if count % 10 == 0:
#             #     print('epoch no.{} train no.{}  loss = {}'.format(epoch, count + 1, loss))
#             if (count > 0 and count % save_step == 0) or (len(data) < batch_size):
#                 torch.save({
#                     'epoch': epoch,
#                     'train_no': count,
#                     'model_state_dict': model.state_dict(),
#                     'optimizer_state_dict': optimizer.state_dict(),
#                     'total_train_step': len(train_loader),
#                     'loss': loss
#                 }, save_ckpt_path)
#             count += 1
#             pbar.update(1)
#             pbar.set_postfix_str(f"Loss: {loss.item():.3f} ({np.mean(losses):.3f})")

#             # if count % 1000 == 0:
#             #     print('epoch no.{} train no.{}  loss = {}'.format(epoch, count + 1, loss))
#             # if (count > 0 and count % save_step == 0) or (len(data) < batch_size):
#             #     torch.save({
#             #         'epoch': epoch,
#             #         'train_no': count,
#             #         'model_state_dict': model.state_dict(),
#             #         'total_train_step': len(train_loader),
#             #         'optimizer_state_dict': optimizer.state_dict(),
#             #         'loss': loss
#             #     }, save_ckpt_path)
            
           

# ########################################################
# torch.save({
#     'epoch': epoch,
#     'train_no': count,
#     'model_state_dict': model.state_dict(),
#     'total_train_step': len(train_loader),
#     'optimizer_state_dict': optimizer.state_dict(),
#     'loss': loss
#     }, save_ckpt_path)
# ########################################################